In [1]:
import os
os.chdir("../")
print(os.getcwd())

c:\Users\Amrit\Intilt\Intrade_ai


In [2]:
import pandas as pd
import numpy as np
import datetime
from utils.data_cleaning import get_missing_data_dates, get_continuous_1min_data

In [3]:
min_data = pd.read_csv('Data/Nifty/nifty_1min.csv')
daily_data = pd.read_csv('Data/Nifty/nifty_1day_continous.csv')

In [4]:
min_data = get_continuous_1min_data('Nifty', min_data)

In [51]:
df = min_data.copy()
df.interpolate(method='polynomial', order=2, inplace=True)

# Add noise to the imputed values
noise = np.random.normal(0, 0.02, df.shape)
df += noise

In [52]:
df1 = daily_data.copy()
df1 = df1.set_index('datetime')
df1 = df1[df1.index >= '2011-01-03']

In [56]:
def add_adj_close(min_data, daily_data):
    min_data.index = pd.to_datetime(min_data.index)
    daily_data.index = pd.to_datetime(daily_data.index)
    min_data['day'] = min_data.index.normalize()
    gp = min_data.groupby('day')
    dfList = []
    for k, res in gp:
        # Check if date is present in the daily data
        if k in daily_data.index:
            # Get the close value of the same date from daily data
            close_value = daily_data.at[k, 'close']
            # get the last row of the group
            last_row = res.iloc[-1]
            # Update the adj close value of the last time of same date with close value from daily data
            res.at[last_row.name, 'adj close'] = close_value
        dfList.append(res)
    min_data = pd.concat(dfList)
    min_data['adj close'].fillna(min_data['close'], inplace=True)
    min_data = min_data.drop(columns=['day'])
    return min_data

In [58]:
temp = add_adj_close(df, df1)

In [60]:
temp

,open,high,low,close,adj close
datetime,,,,,
2011-01-03 09:15:00,6178.238576,6178.522562,6171.724723,6172.068388,6172.068388
2011-01-03 09:16:00,6173.784804,6173.821487,6168.973980,6170.842889,6170.842889
2011-01-03 09:17:00,6171.513909,6174.620438,6170.989161,6174.669041,6174.669041
2011-01-03 09:18:00,6174.008973,6178.193365,6174.028453,6177.705282,6177.705282
2011-01-03 09:19:00,6177.822029,6178.243138,6176.748200,6177.048580,6177.048580
...,...,...,...,...,...
2021-12-31 15:25:00,17357.918089,17358.466220,17353.783842,17356.302035,17356.302035
2021-12-31 15:26:00,17356.121732,17359.696404,17355.614759,17359.321014,17359.321014
2021-12-31 15:27:00,17358.791325,17360.501200,17356.230626,17359.803388,17359.803388
